# Text generator using LSTM on project Gutenberg

# Imports

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


### Load file after downloading it 

In [2]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [3]:
type(raw_text)

str

In order to fit the neural network we need to transform characters into integers

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

Let's summarize the dataset

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163781
Total Vocab:  59


We now need to define the training data for the network

Prepare the dataset of input to output pairs encoded as integers

In [6]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163681


Now that we have prepared our training data we need to transform it so that it is suitable for use with Keras.

First we must transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network.

Next we need to rescale the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network that uses the sigmoid activation function by default.

Finally, we need to convert the output patterns (single characters converted to integers) into a one hot encoding.

In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

 Here we define a single hidden LSTM layer with 256 memory units. The network uses dropout with a probability of 20. The output layer is a Dense layer using the softmax activation function to output a probability prediction

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [10]:
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

Epoch 1/5
163681/163681 [==============================] - 415s 3ms/step - loss: 2.9842

Epoch 00001: loss improved from inf to 2.98421, saving model to weights-improvement-01-2.9842.hdf5
Epoch 2/5
163681/163681 [==============================] - 417s 3ms/step - loss: 2.7973

Epoch 00002: loss improved from 2.98421 to 2.79728, saving model to weights-improvement-02-2.7973.hdf5
Epoch 3/5
163681/163681 [==============================] - 423s 3ms/step - loss: 2.7113

Epoch 00003: loss improved from 2.79728 to 2.71133, saving model to weights-improvement-03-2.7113.hdf5
Epoch 4/5
163681/163681 [==============================] - 428s 3ms/step - loss: 2.6402

Epoch 00004: loss improved from 2.71133 to 2.64023, saving model to weights-improvement-04-2.6402.hdf5
Epoch 5/5
163681/163681 [==============================] - 421s 3ms/step - loss: 2.5812

Epoch 00005: loss improved from 2.64023 to 2.58122, saving model to weights-improvement-05-2.5812.hdf5


In [11]:
# load the network weights
filename = "weights-improvement-05-2.5812.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [13]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Seed:")? (<ipython-input-13-e78b59957073>, line 4)